# Gorilla Data Engineer Assessment
## Transportation distribution charge

Author: Nicolás Arturo Cozzi Elzo 

Date: 3/20/2022

In [461]:
import pandas as pd
import numpy as np
import datetime as dt
import random
import math

from time import time # for benchmark

In [462]:
# reading excel file:
filepath = 'C://Users/ncozz/Documents/gorilla_test_data.xlsx' # feel free to change path of the file
data_excel = pd.ExcelFile(filepath)
sheet_names = data_excel.sheet_names
sheet_names

['meter_list', 'forecast_table', 'rate_table']

In [463]:
# generating dataframes for each sheet of the .xlsx file:
meter_list = pd.read_excel(filepath, sheet_name=sheet_names[0])
meter_list

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [464]:
forecast_table = pd.read_excel(filepath, sheet_name=sheet_names[1])
forecast_table

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [465]:
rate_table = pd.read_excel(filepath, sheet_name=sheet_names[2])
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537
1136,2024-10-01,WM2,732000,NaN,0.7534
1137,2024-10-01,WM3,0,73200.0,0.7263
1138,2024-10-01,WM3,73200,732000.0,0.6109


--------------------
### Task 1: total cost & consumption dataframes
#### Total consumption per meter:
From what I understand, it is simply the total sum of forecasted kwh for each meter_id - which results in the same table as the first sheet in the Excel spreadsheet.

In [466]:
# total consumption per meter:
total_consumption = forecast_table.groupby(['meter_id'], as_index=False).sum()
total_consumption.kwh = total_consumption.kwh.round(2)
total_consumption

,meter_id,kwh
0,14676236,28978.0
1,34509937,78324.0
2,50264822,265667.0
3,88357331,484399.0


#### Total cost per meter:
For this, the steps are:
* Filter the exit_zone that concern the observed meter_id, in the rate table
* Match the rates per kwh with the actual forecasted value, for each value range and each date range
* Match the meter_id with the respective zone in the rate table

In [467]:
# total cost per meter:

rate_table_filter = rate_table[rate_table['exit_zone'].isin(meter_list.exit_zone)] # filtering exit zones for observed id's
rate_table_filter = pd.merge(rate_table_filter, meter_list,
                               how='left',
                               left_on=['exit_zone'], right_on=['exit_zone'])
rate_table_filter.aq_max_kwh = rate_table_filter.aq_max_kwh.fillna(99999999999) # filling missing values

# filtering so that we obtain the range matching the aq_kwh:
rate_table_filter = rate_table_filter[(rate_table_filter.aq_kwh >= rate_table_filter.aq_min_kwh) &
                                      (rate_table_filter.aq_kwh <= rate_table_filter.aq_max_kwh)]
rate_table_filter

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,meter_id,aq_kwh
0,2020-04-01,EA1,0,73200.0,0.2652,14676236,28978
4,2020-04-01,NT1,73200,732000.0,0.2181,50264822,265667
7,2020-04-01,SE2,73200,732000.0,0.2318,88357331,484399
10,2020-04-01,SO1,73200,732000.0,0.2743,34509937,78324
12,2020-10-01,EA1,0,73200.0,0.2970,14676236,28978
16,2020-10-01,NT1,73200,732000.0,0.2443,50264822,265667
19,2020-10-01,SE2,73200,732000.0,0.2596,88357331,484399
22,2020-10-01,SO1,73200,732000.0,0.3072,34509937,78324
24,2021-04-01,EA1,0,73200.0,0.3327,14676236,28978
28,2021-04-01,NT1,73200,732000.0,0.2736,50264822,265667


In [468]:
# creating date bins for matching rate values per meter_id:
df_bins = pd.DataFrame(data={'lower': sorted(rate_table_filter.date.unique()),
                             'upper': sorted(rate_table_filter.date.unique())[1:] + [dt.date(2100, 12, 31)]})
bins = pd.to_datetime(np.insert(df_bins.upper.values, 0, df_bins.lower.iat[0]), format='%Y%b%d')
bins

DatetimeIndex(['2020-04-01', '2020-10-01', '2021-04-01', '2021-10-01',
               '2022-04-01', '2022-10-01', '2023-04-01', '2023-10-01',
               '2024-04-01', '2024-10-01', '2100-12-31'],
              dtype='datetime64[ns]', freq=None)

In [469]:
forecast_table_cost = forecast_table.copy()
forecast_table_cost['range'] = pd.cut(forecast_table_cost.date, bins=bins, right=False).apply(lambda x: x.left)
forecast_table_cost.range = forecast_table_cost.range.astype('datetime64')
forecast_table_cost = forecast_table_cost.drop('date', axis=1).groupby(['meter_id','range'], as_index=False).sum()

# merging dataframes:
forecast_table_cost = pd.merge(forecast_table_cost, rate_table_filter,
                               how='left', left_on=['meter_id','range'], right_on=['meter_id','date']).drop(
    ['date','aq_min_kwh','aq_max_kwh','aq_kwh'], axis=1).rename(
    columns={'range':'date'})

forecast_table_cost

# we observe that forecasts do not go beyond 2022

,meter_id,date,kwh,exit_zone,rate_p_per_kwh
0,14676236,2020-04-01,1769.354787,EA1,0.2652
1,14676236,2020-10-01,8565.582872,EA1,0.2970
2,14676236,2021-04-01,4717.664057,EA1,0.3327
3,14676236,2021-10-01,8512.419037,EA1,0.3726
4,14676236,2022-04-01,5375.856768,EA1,0.4173
5,14676236,2022-10-01,37.122479,EA1,0.4674
6,34509937,2020-04-01,8217.361143,SO1,0.2743
7,34509937,2020-10-01,24558.935299,SO1,0.3072
8,34509937,2021-04-01,9900.663301,SO1,0.3441
9,34509937,2021-10-01,22644.929722,SO1,0.3854


In [470]:
# creating column with total cost, converted to pounds:
forecast_table_cost['total_cost_pounds'] = forecast_table_cost.kwh * forecast_table_cost.rate_p_per_kwh * 0.01
forecast_table_cost

,meter_id,date,kwh,exit_zone,rate_p_per_kwh,total_cost_pounds
0,14676236,2020-04-01,1769.354787,EA1,0.2652,4.692329
1,14676236,2020-10-01,8565.582872,EA1,0.2970,25.439781
2,14676236,2021-04-01,4717.664057,EA1,0.3327,15.695668
3,14676236,2021-10-01,8512.419037,EA1,0.3726,31.717273
4,14676236,2022-04-01,5375.856768,EA1,0.4173,22.433450
5,14676236,2022-10-01,37.122479,EA1,0.4674,0.173510
6,34509937,2020-04-01,8217.361143,SO1,0.2743,22.540222
7,34509937,2020-10-01,24558.935299,SO1,0.3072,75.445049
8,34509937,2021-04-01,9900.663301,SO1,0.3441,34.068182
9,34509937,2021-10-01,22644.929722,SO1,0.3854,87.273559


In [471]:
# final dataframe:
forecast_table_final = forecast_table_cost.groupby(['meter_id'], as_index=False).sum().drop('rate_p_per_kwh',axis=1)
forecast_table_final.kwh = forecast_table_final.kwh.round(2)
forecast_table_final.total_cost_pounds = forecast_table_final.total_cost_pounds.round(2)
forecast_table_final = forecast_table_final.rename(columns={'kwh':'total_kwh'})

forecast_table_final

,meter_id,total_kwh,total_cost_pounds
0,14676236,28978.0,100.15
1,34509937,78324.0,275.49
2,50264822,265667.0,731.24
3,88357331,484399.0,1433.16


--------------------------
### Task 2: random meter generator

In [472]:
# to generate a random annual quantity, we generate a statistics-based number
# we generate a strictly positive random number with mean according to the original meter list
# thus, we implement a log-normal distribution for the random number generator

def random_aq_kwh(df = meter_list):
    return math.exp(np.random.normal(loc = np.log(df.aq_kwh).mean(),
                                   scale = np.log(df.aq_kwh).var()))

In [473]:
# function for generating a random exit zone:

def random_exit_zone(exit_zone_list = rate_table.exit_zone.unique()):
    return random.choice(exit_zone_list)

In [474]:
# function for generating a random id:

def random_meter_id(id_list = meter_list.meter_id):
    random_number = random.randint(10000001,99999999)
    # loop for ensuring that id is not in original meter_id list:
    if random_number in id_list:
        while random_number in id_list:
            random_number = random.randint(10000001,99999999)
    return random_number

In [475]:
# function for generating random meter:

def random_meter_generator(id_list = meter_list.meter_id):
    random_meter = {'meter_id': random_meter_id(),
                    'exit_zone': random_exit_zone(),
                    'aq_kwh': random_aq_kwh()}
    return pd.Series(random_meter)

In [476]:
# testing our new function:
random_meter_generator()

meter_id          61262700
exit_zone               LT
aq_kwh       827432.561178
dtype: object

------------------------
### Task 3: mock consumption data

In [477]:
def mock_consumption_data(meter_list = meter_list.meter_id,
                         start_date = dt.date(2020, 4,1),
                         duration=1):
    
    meter_list_len = len(meter_list)
    # list of dates, repeated for each meter_id
    list_dates = [start_date + dt.timedelta(days=x) for x in range(duration)] * meter_list_len
    list_dates = pd.to_datetime(list_dates)
    
    #list of each meter_id repeated by duration days
    meter_list_full = [y for x in meter_list for y in (x,)*duration]
    
    # random consumption and rate lists:
    random_consumption = np.random.uniform(0, 1000, duration*meter_list_len).round(2)
    
    # final dataframe:
    d = {'meter_id': meter_list_full,
        'date': list_dates,
        'kwh': random_consumption}
    return pd.DataFrame(d)

In [478]:
# testing the function:
mock_consumption_data(duration=4)

,meter_id,date,kwh
0,14676236,2020-04-01,596.15
1,14676236,2020-04-02,323.58
2,14676236,2020-04-03,720.29
3,14676236,2020-04-04,349.77
4,34509937,2020-04-01,664.11
5,34509937,2020-04-02,338.33
6,34509937,2020-04-03,198.32
7,34509937,2020-04-04,6.11
8,50264822,2020-04-01,13.83
9,50264822,2020-04-02,444.56


-------------------
### Task 4: transportation cost table function

In [479]:
# for this function, as suggested, we take the logic from task 1

def transportation_cost_calc(id_list = meter_list.meter_id ,meter_list = meter_list,
                             forecast_table = forecast_table, rate_table = rate_table):
    
    meter_list = meter_list[meter_list.meter_id.isin(id_list)]
    
    # total cost per meter:
    rate_table_filter = rate_table[rate_table['exit_zone'].isin(meter_list.exit_zone)] # filtering exit zones for observed id's
    rate_table_filter = pd.merge(rate_table_filter, meter_list,
                               how='left',
                               left_on=['exit_zone'], right_on=['exit_zone'])
    rate_table_filter.aq_max_kwh = rate_table_filter.aq_max_kwh.fillna(99999999999) # filling missing values
    rate_table_filter = rate_table_filter[(rate_table_filter.aq_kwh >= rate_table_filter.aq_min_kwh) &
                                      (rate_table_filter.aq_kwh <= rate_table_filter.aq_max_kwh)]
    
    # creating date bins for matching rate values per meter_id:
    df_bins = pd.DataFrame(data={'lower': sorted(rate_table_filter.date.unique()),
                                 'upper': sorted(rate_table_filter.date.unique())[1:] + [dt.date(2100, 12, 31)]})
    bins = pd.to_datetime(np.insert(df_bins.upper.values, 0, df_bins.lower.iat[0]), format='%Y%b%d')
    
    forecast_table_cost = forecast_table
    forecast_table_cost['range'] = pd.cut(forecast_table_cost.date, bins=bins, right=False).apply(lambda x: x.left)
    forecast_table_cost.range = forecast_table_cost.range.astype('datetime64')
    forecast_table_cost = forecast_table_cost.drop('date', axis=1).groupby(['meter_id','range'], as_index=False).sum()

    # merging dataframes:
    forecast_table_cost = pd.merge(forecast_table_cost, rate_table_filter,
                                   how='left', left_on=['meter_id','range'], right_on=['meter_id','date']).drop(
        ['date','aq_min_kwh','aq_max_kwh','aq_kwh'], axis=1).rename(
        columns={'range':'date'})
    forecast_table_cost['total_cost_pounds'] = forecast_table_cost.kwh * forecast_table_cost.rate_p_per_kwh * 0.01
    
    # final dataframe:
    forecast_table_cost = forecast_table_cost.groupby(['meter_id'], as_index=False).sum().drop('rate_p_per_kwh',axis=1)
    forecast_table_cost.kwh = forecast_table_cost.kwh.round(2)
    forecast_table_cost.total_cost_pounds = forecast_table_cost.total_cost_pounds.round(2)
    forecast_table_cost = forecast_table_cost.rename(columns={'kwh':'total_kwh'})
    return forecast_table_cost

In [480]:
start = time()
transportation_cost_calc()
print(f'Time to run: {time() - start} seconds')

Time to run: 0.03425908088684082 seconds


#### Benchmark using larger lists

In [481]:
# let's benchmark against a list of meters twice as large:

meter_list_2 = meter_list.copy()
meter_list_2 = meter_list_2.append(random_meter_generator(), ignore_index=True)
meter_list_2 = meter_list_2.append(random_meter_generator(), ignore_index=True)
meter_list_2 = meter_list_2.append(random_meter_generator(), ignore_index=True)
meter_list_2 = meter_list_2.append(random_meter_generator(), ignore_index=True)
meter_list_2

,meter_id,aq_kwh,exit_zone
0,14676236,28978.000000,EA1
1,34509937,78324.000000,SO1
2,50264822,265667.000000,NT1
3,88357331,484399.000000,SE2
4,25393376,344984.733735,SC4
5,11194498,25314.377069,NT3
6,25293883,33029.311468,NE2
7,19540663,337314.452606,EA4


In [482]:
# and a forecast table to match:
forecast_table_2 = mock_consumption_data(meter_list=meter_list_2.meter_id, duration=int(len(forecast_table)/4))
forecast_table_2

,meter_id,date,kwh
0,14676236,2020-04-01,72.74
1,14676236,2020-04-02,996.58
2,14676236,2020-04-03,219.38
3,14676236,2020-04-04,322.63
4,14676236,2020-04-05,822.21
...,...,...,...
6819,19540663,2022-07-28,113.76
6820,19540663,2022-07-29,444.01
6821,19540663,2022-07-30,53.12
6822,19540663,2022-07-31,51.39


In [483]:
start = time()
transportation_cost_calc(id_list=meter_list_2.meter_id ,meter_list = meter_list_2, forecast_table = forecast_table_2)
print(f'Time to run: {time() - start} seconds')

Time to run: 0.03900146484375 seconds


In [484]:
transportation_cost_calc(id_list=meter_list_2.meter_id ,meter_list = meter_list_2, forecast_table = forecast_table_2)

,meter_id,total_kwh,total_cost_pounds
0,11194498,432839.71,901.74
1,14676236,429963.21,1424.74
2,19540663,412334.40,1512.22
3,25293883,414167.59,1115.45
4,25393376,426759.54,1221.69
5,34509937,422899.15,1449.71
6,50264822,418206.06,1133.03
7,88357331,426584.91,1241.36


In [485]:
# even larger dataframe:

meter_list_3 = meter_list.copy()
for i in range(1000):
    meter_list_3 = meter_list_3.append(random_meter_generator(id_list = meter_list_3.meter_id), ignore_index=True)
meter_list_3.shape

(1004, 3)

In [486]:
forecast_table_3 = mock_consumption_data(meter_list=meter_list_3.meter_id, duration=int(len(forecast_table)/4))
forecast_table_3.shape

(856412, 3)

In [487]:
start = time()
transportation_cost_calc(id_list=meter_list_3.meter_id ,meter_list = meter_list_3, forecast_table = forecast_table_3)
print(f'Time to run: {time() - start} seconds')

Time to run: 0.1650240421295166 seconds


In [488]:
transportation_cost_calc(id_list=meter_list_3.meter_id ,meter_list = meter_list_3, forecast_table = forecast_table_3)

,meter_id,total_kwh,total_cost_pounds
0,10061209,428442.46,1241.70
1,10119631,420874.38,1409.23
2,10497845,428355.09,1739.11
3,10525905,447792.98,1345.64
4,10916141,439748.67,1019.21
...,...,...,...
999,99204551,430541.77,1130.20
1000,99360959,435992.28,1278.36
1001,99421313,417098.99,1395.54
1002,99707214,434331.25,1525.34


----------------------------
Considering the amount of time invested for the solution, speed is satisfactory in my opinion. </br>
There seems to be a basis of time that the function needs to operate properly (around 0.03 seconds); beyond that, increase in time requirements are marginal. </br>
</br>
The result needs more robustness checks - particularly when handling different formats of data, different entries, and extreme cases of repeated observations for exit zones. Due to time constraints, these are done in a shallow manner.
----------------
Needless to say, thank you very much for your time if you made it here! I don't have experience reviewing python codes for applicants, but I'm sure it takes awhile